In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [7]:
df = pd.read_csv(r"C:\Users\kanha\Desktop\Sap_assignment\data\ner_dataset.csv", encoding = "ISO-8859-1")

In [8]:
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [9]:
df['Sentence #'] = df['Sentence #'].ffill()

# Group by sentence
sentences = []
for _, group in df.groupby('Sentence #'):
    sentence = list(zip(group['Word'], group['POS'], group['Tag']))
    sentences.append(sentence)

In [11]:
# Total number of tokens (words)
total_tokens = len(df)

# Total number of sentences
total_sentences = df['Sentence #'].nunique()

# Number of unique tags (NER labels)
unique_tags = df['Tag'].unique()
num_unique_tags = len(unique_tags)

# Number of unique words
num_unique_words = df['Word'].nunique()

# Number of unique POS tags
num_unique_pos = df['POS'].nunique()

# Distribution of entity tags
tag_counts = df['Tag'].value_counts()

# Top 10 most frequent words
top_words = df['Word'].value_counts().head(10)

# Average sentence length (in tokens)
avg_sentence_len = df.groupby('Sentence #')['Word'].count().mean()

# Max sentence length
max_sentence_len = df.groupby('Sentence #')['Word'].count().max()

# Min sentence length
min_sentence_len = df.groupby('Sentence #')['Word'].count().min()

# Count of entities (excluding "O")
entity_only = df[df['Tag'] != 'O']['Tag'].value_counts()

# Example tagged sentence
sample_sentence = df[df['Sentence #'] == df['Sentence #'].unique()[0]][['Word', 'Tag']]


In [12]:
print(f"1. Total tokens: {total_tokens}")
print(f"2. Total sentences: {total_sentences}")
print(f"3. Unique NER tags: {unique_tags}")
print(f"4. Number of unique tags: {num_unique_tags}")
print(f"5. Number of unique words: {num_unique_words}")
print(f"6. Number of unique POS tags: {num_unique_pos}")
print(f"7. Tag distribution:\n{tag_counts}")
print(f"8. Top 10 most frequent words:\n{top_words}")
print(f"9. Average sentence length: {avg_sentence_len:.2f} tokens")
print(f"10. Max sentence length: {max_sentence_len}")
print(f"11. Min sentence length: {min_sentence_len}")
print(f"12. Named Entity Tag counts (excluding 'O'):\n{entity_only}")
print(f"13. Sample tagged sentence:\n{sample_sentence}")


1. Total tokens: 1048575
2. Total sentences: 47959
3. Unique NER tags: ['O' 'B-geo' 'B-gpe' 'B-per' 'I-geo' 'B-org' 'I-org' 'B-tim' 'B-art'
 'I-art' 'I-per' 'I-gpe' 'I-tim' 'B-nat' 'B-eve' 'I-eve' 'I-nat']
4. Number of unique tags: 17
5. Number of unique words: 35177
6. Number of unique POS tags: 42
7. Tag distribution:
Tag
O        887908
B-geo     37644
B-tim     20333
B-org     20143
I-per     17251
B-per     16990
I-org     16784
B-gpe     15870
I-geo      7414
I-tim      6528
B-art       402
B-eve       308
I-art       297
I-eve       253
B-nat       201
I-gpe       198
I-nat        51
Name: count, dtype: int64
8. Top 10 most frequent words:
Word
the    52573
.      47761
,      32754
of     26354
in     26323
to     23213
a      20481
and    19936
The    11313
's     10923
Name: count, dtype: int64
9. Average sentence length: 21.86 tokens
10. Max sentence length: 104
11. Min sentence length: 1
12. Named Entity Tag counts (excluding 'O'):
Tag
B-geo    37644
B-tim    20333
B-org   